<a href="https://colab.research.google.com/github/SamuelMiller413/N1-Health/blob/main/N1_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Set Up

#### Imports

In [ ]:
import numpy as np 
import pandas as pd 

                                                                          # PRE-PROCESSING
from sklearn.preprocessing import StandardScaler
                                                                          # FEATURE SELECTION
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel, mutual_info_regression, RFE, RFECV

                                                                          # PIPELINE
from sklearn.pipeline import Pipeline
                                                                          # REGRESSORS       
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
                                          
                                                                          # NEURAL NETWORK
from sklearn.neural_network import MLPRegressor

                                                                          # CROSS VALIDATION
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit, StratifiedKFold
from sklearn.model_selection import learning_curve, cross_val_predict
from sklearn.model_selection import KFold, RandomizedSearchCV

                                                                          # EVALUATION
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import SCORERS

                                                                          # PLOTTING
import random
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

                                                                          # DATA SET



#### Drive Mount

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Data Load

##### FDA

The Food Atlas data span multiple CSV's:
* primary dataset
* variable list
* 2 supplemental datasets 
    * county
    * state
<br/>  

I'll be supplementing the primary
dataset with the county dataset and using the variable list for reference while merging and parsing the dataframes.

In [ ]:
# path to primary csv
fda = '/content/drive/MyDrive/N1 Health/N1 Health/Datasets//USDA ERS/StateAndCountyData.csv'

# path to variable list
fda_vars = '/content/drive/MyDrive/N1 Health/N1 Health/Datasets/USDA ERS/VariableList.csv'

# supplemental FDA paths
fda_county = '/content/drive/MyDrive/N1 Health/N1 Health/Datasets/USDA ERS/SupplementalDataCounty.csv'
# fda_state = '/content/drive/MyDrive/N1 Health/N1 Health/Datasets/USDA ERS/SupplementalDataState.csv'

# read to df
df_fda_full = pd.read_csv(fda)
df_fda_vars = pd.read_csv(fda_vars)
df_fda_county = pd.read_csv(fda_county)
# df_fda_state = pd.read_csv(fda_state)

##### CDC

In [ ]:
## path
# cdc = '/content/drive/MyDrive/N1 Health/N1 Health/Datasets/CDC/500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2019_release.csv'

## read to df
# df_cdc = pd.read_csv(cdc)

##### CMS

In [ ]:
# path
# cms = '/content/drive/MyDrive/N1 Health/N1 Health/Datasets/CMS/Geographic Variation Public Use File State County.csv'

# read to df
# df_cms = pd.read_csv(cms)

### Initial EDA

#### df_fda_vars

In [ ]:
df_fda_vars

,Variable_Name,Category_Name,Category_Code,Subcategory_Name,Variable_Code,Geography,Units
0,"Population, low access to store, 2010",Access and Proximity to Grocery Store,ACCESS,Overall,LACCESS_POP10,CNTY10,Count
1,"Population, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Overall,LACCESS_POP15,CNTY10,Count
2,"Population, low access to store (% change), 20...",Access and Proximity to Grocery Store,ACCESS,Overall,PCH_LACCESS_POP_10_15,CNTY10,% change
3,"Population, low access to store (%), 2010",Access and Proximity to Grocery Store,ACCESS,Overall,PCT_LACCESS_POP10,CNTY10,Percent
4,"Population, low access to store (%), 2015",Access and Proximity to Grocery Store,ACCESS,Overall,PCT_LACCESS_POP15,CNTY10,Percent
...,...,...,...,...,...,...,...
276,"Persistent-poverty counties, 2010",Socioeconomic Characteristics,SOCIOECONOMIC,Income Level,PERPOV10,CNTY10,Legend
277,"Child poverty rate, 2015",Socioeconomic Characteristics,SOCIOECONOMIC,Income Level,CHILDPOVRATE15,CNTY10,Percent
278,"Persistent-child-poverty counties, 2010",Socioeconomic Characteristics,SOCIOECONOMIC,Income Level,PERCHLDPOV10,CNTY10,Legend
279,"Metro/nonmetro counties, 2010",Socioeconomic Characteristics,SOCIOECONOMIC,Other,METRO13,CNTY10,Legend


In [ ]:
df_fda_vars.Category_Code.unique().tolist()

['ACCESS',
 'STORES',
 'RESTAURANTS',
 'ASSISTANCE',
 'INSECURITY',
 'PRICES_TAXES',
 'LOCAL',
 'HEALTH',
 'SOCIOECONOMIC']

In [ ]:
def cat_var(df):
    '''
    Creates adictionary of variables for each category
    in the variable list df.
    '''
    cat_list = df.Category_Code.unique().tolist()
    all_cats = {}
    for i in range(len(cat_list)):
        all_cats[cat_list[i]] = df[df.Category_Code == cat_list[i]].Variable_Code.tolist()
    for k,v in all_cats.items():
        print(f'{k}:')
        print(v)
        print('')
    return all_cats




In [ ]:
all_cats = cat_var(df_fda_vars)

ACCESS:
['LACCESS_POP10', 'LACCESS_POP15', 'PCH_LACCESS_POP_10_15', 'PCT_LACCESS_POP10', 'PCT_LACCESS_POP15', 'LACCESS_LOWI10', 'LACCESS_LOWI15', 'PCH_LACCESS_LOWI_10_15', 'PCT_LACCESS_LOWI10', 'PCT_LACCESS_LOWI15', 'LACCESS_HHNV10', 'LACCESS_HHNV15', 'PCH_LACCESS_HHNV_10_15', 'PCT_LACCESS_HHNV10', 'PCT_LACCESS_HHNV15', 'LACCESS_SNAP15', 'PCT_LACCESS_SNAP15', 'LACCESS_CHILD10', 'LACCESS_CHILD15', 'PCH_LACCESS_CHILD_10_15', 'PCT_LACCESS_CHILD10', 'PCT_LACCESS_CHILD15', 'LACCESS_SENIORS10', 'LACCESS_SENIORS15', 'PCH_LACCESS_SENIORS_10_15', 'PCT_LACCESS_SENIORS10', 'PCT_LACCESS_SENIORS15', 'LACCESS_WHITE15', 'PCT_LACCESS_WHITE15', 'LACCESS_BLACK15', 'PCT_LACCESS_BLACK15', 'LACCESS_HISP15', 'PCT_LACCESS_HISP15', 'LACCESS_NHASIAN15', 'PCT_LACCESS_NHASIAN15', 'LACCESS_NHNA15', 'PCT_LACCESS_NHNA15', 'LACCESS_NHPI15', 'PCT_LACCESS_NHPI15', 'LACCESS_MULTIR15', 'PCT_LACCESS_MULTIR15']

STORES:
['GROC11', 'GROC16', 'PCH_GROC_11_16', 'GROCPTH11', 'GROCPTH16', 'PCH_GROCPTH_11_16', 'SUPERC11', 'SUPE

In [ ]:
print(all_cats.keys())

dict_keys(['ACCESS', 'STORES', 'RESTAURANTS', 'ASSISTANCE', 'INSECURITY', 'PRICES_TAXES', 'LOCAL', 'HEALTH', 'SOCIOECONOMIC'])


In [53]:
# select categories relevant to the project
cat_select = ['ACCESS','ASSISTANCE', 'INSECURITY','HEALTH', 'SOCIOECONOMIC']
print(cat_select)

['ACCESS', 'ASSISTANCE', 'INSECURITY', 'HEALTH', 'SOCIOECONOMIC']


#### df_fda_county

In [ ]:
df_fda_county

,FIPS,State,County,Variable_Code,Value
0,1001,AL,Autauga County,2010_Census_Population,54571
1,1001,AL,Autauga County,Population_Estimate_2011,55208
2,1001,AL,Autauga County,Population_Estimate_2012,54936
3,1001,AL,Autauga County,Population_Estimate_2013,54713
4,1001,AL,Autauga County,Population_Estimate_2014,54876
...,...,...,...,...,...
28273,56045,WY,Weston County,Population_Estimate_2014,7138
28274,56045,WY,Weston County,Population_Estimate_2015,7197
28275,56045,WY,Weston County,Population_Estimate_2016,7213
28276,56045,WY,Weston County,Population_Estimate_2017,6986


In [ ]:
# shape
print(f'Shape of df_fda_county: {df_fda_county.shape}')

print('')

# columns
print(f'Columns in df_fda_county: {df_fda_county.columns}')

Shape of df_fda_county: (28278, 5)

Columns in df_fda_county: Index(['FIPS', 'State', 'County', 'Variable_Code', 'Value'], dtype='object')


In [ ]:
# display
df_fda_county.head()

,FIPS,State,County,Variable_Code,Value
0,1001,AL,Autauga County,2010_Census_Population,54571
1,1001,AL,Autauga County,Population_Estimate_2011,55208
2,1001,AL,Autauga County,Population_Estimate_2012,54936
3,1001,AL,Autauga County,Population_Estimate_2013,54713
4,1001,AL,Autauga County,Population_Estimate_2014,54876


In [ ]:
# pivot
df_fda_county = df_fda_county.pivot_table(values='Value', index=['FIPS', 'State', 'County'], columns='Variable_Code')
df_fda_county.reset_index(inplace=True)

In [ ]:
# display
df_fda_county.head()

Variable_Code,FIPS,State,County,2010_Census_Population,Population_Estimate_2011,Population_Estimate_2012,Population_Estimate_2013,Population_Estimate_2014,Population_Estimate_2015,Population_Estimate_2016,Population_Estimate_2017,Population_Estimate_2018
0,1001,AL,Autauga County,54571,55208,54936,54713,54876,54838,55242,55443,55601
1,1003,AL,Baldwin County,182265,186540,190143,194886,199189,202995,207712,212619,218022
2,1005,AL,Barbour County,27457,27350,27174,26944,26758,26294,25819,25158,24881
3,1007,AL,Bibb County,22915,22747,22664,22516,22541,22562,22576,22555,22400
4,1009,AL,Blount County,57322,57554,57570,57611,57521,57522,57517,57827,57840


In [ ]:
# shape
print(f'Shape of df_fda_county: {df_fda_county.shape}')

print('')

# columns
print(f'Columns in df_fda_county: {df_fda_county.columns}')

Shape of df_fda_county: (3142, 12)

Columns in df_fda_county: Index(['FIPS', 'State', 'County', '2010_Census_Population',
       'Population_Estimate_2011', 'Population_Estimate_2012',
       'Population_Estimate_2013', 'Population_Estimate_2014',
       'Population_Estimate_2015', 'Population_Estimate_2016',
       'Population_Estimate_2017', 'Population_Estimate_2018'],
      dtype='object', name='Variable_Code')


In [ ]:
# info
df_fda_county.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   FIPS                      3142 non-null   int64 
 1   State                     3142 non-null   object
 2   County                    3142 non-null   object
 3   2010_Census_Population    3142 non-null   int64 
 4   Population_Estimate_2011  3142 non-null   int64 
 5   Population_Estimate_2012  3142 non-null   int64 
 6   Population_Estimate_2013  3142 non-null   int64 
 7   Population_Estimate_2014  3142 non-null   int64 
 8   Population_Estimate_2015  3142 non-null   int64 
 9   Population_Estimate_2016  3142 non-null   int64 
 10  Population_Estimate_2017  3142 non-null   int64 
 11  Population_Estimate_2018  3142 non-null   int64 
dtypes: int64(10), object(2)
memory usage: 294.7+ KB


In [ ]:
# describe
df_fda_county.describe()

Variable_Code,FIPS,2010_Census_Population,Population_Estimate_2011,Population_Estimate_2012,Population_Estimate_2013,Population_Estimate_2014,Population_Estimate_2015,Population_Estimate_2016,Population_Estimate_2017,Population_Estimate_2018
count,3142.000000,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03
mean,30383.649268,9.826204e+04,9.916614e+04,9.989631e+04,1.005913e+05,1.013324e+05,1.020823e+05,1.028235e+05,1.034841e+05,1.041271e+05
std,15162.508374,3.129467e+05,3.163954e+05,3.195096e+05,3.224247e+05,3.253350e+05,3.281943e+05,3.305395e+05,3.320993e+05,3.334863e+05
min,1001.000000,8.200000e+01,9.000000e+01,8.600000e+01,8.900000e+01,8.900000e+01,8.800000e+01,8.800000e+01,8.800000e+01,8.800000e+01
25%,18177.500000,1.111450e+04,1.108700e+04,1.100525e+04,1.101700e+04,1.098550e+04,1.094800e+04,1.093525e+04,1.088900e+04,1.092650e+04
50%,29176.000000,2.587200e+04,2.579450e+04,2.578850e+04,2.571050e+04,2.564200e+04,2.563250e+04,2.571650e+04,2.578400e+04,2.575850e+04
75%,45080.500000,6.678000e+04,6.699225e+04,6.711200e+04,6.740875e+04,6.760700e+04,6.748825e+04,6.730625e+04,6.781575e+04,6.782050e+04
max,56045.000000,9.818605e+06,9.876727e+06,9.938436e+06,9.998105e+06,1.004841e+07,1.009704e+07,1.012054e+07,1.011876e+07,1.010552e+07


In [ ]:
df_fda_county.isnull().sum()

Variable_Code
FIPS                        0
State                       0
County                      0
2010_Census_Population      0
Population_Estimate_2011    0
Population_Estimate_2012    0
Population_Estimate_2013    0
Population_Estimate_2014    0
Population_Estimate_2015    0
Population_Estimate_2016    0
Population_Estimate_2017    0
Population_Estimate_2018    0
dtype: int64

In [ ]:
df_fda_county

Variable_Code,FIPS,State,County,2010_Census_Population,Population_Estimate_2011,Population_Estimate_2012,Population_Estimate_2013,Population_Estimate_2014,Population_Estimate_2015,Population_Estimate_2016,Population_Estimate_2017,Population_Estimate_2018
0,1001,AL,Autauga County,54571,55208,54936,54713,54876,54838,55242,55443,55601
1,1003,AL,Baldwin County,182265,186540,190143,194886,199189,202995,207712,212619,218022
2,1005,AL,Barbour County,27457,27350,27174,26944,26758,26294,25819,25158,24881
3,1007,AL,Bibb County,22915,22747,22664,22516,22541,22562,22576,22555,22400
4,1009,AL,Blount County,57322,57554,57570,57611,57521,57522,57517,57827,57840
...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,WY,Sweetwater County,43806,43981,45005,45162,44957,44754,44275,43547,43051
3138,56039,WY,Teton County,21294,21414,21625,22318,22777,23016,23161,23261,23081
3139,56041,WY,Uinta County,21118,20893,20994,20953,20827,20770,20691,20456,20299
3140,56043,WY,Washakie County,8533,8448,8408,8414,8275,8280,8168,8035,7885


In [ ]:
col_fda_county = df_fda_county.columns.tolist()
for col in col_fda_county:
    print(col)

FIPS
State
County
2010_Census_Population
Population_Estimate_2011
Population_Estimate_2012
Population_Estimate_2013
Population_Estimate_2014
Population_Estimate_2015
Population_Estimate_2016
Population_Estimate_2017
Population_Estimate_2018


#### df_fda_full

In [ ]:
# shape
print(f'Shape of df_fda_full: {df_fda_full.shape}')

print('')

# columns
print(f'Columns in df_fda_full: {df_fda.columns}')

Shape of df_fda: (852810, 5)

Columns in df_fda: Index(['FIPS', 'State', 'County', 'Variable_Code', 'Value'], dtype='object')


In [ ]:
# display
df_fda_full.head()

,FIPS,State,County,Variable_Code,Value
0,1001,AL,Autauga,LACCESS_POP10,18428.439690
1,1001,AL,Autauga,LACCESS_POP15,17496.693040
2,1001,AL,Autauga,PCH_LACCESS_POP_10_15,-5.056026
3,1001,AL,Autauga,PCT_LACCESS_POP10,33.769657
4,1001,AL,Autauga,PCT_LACCESS_POP15,32.062255


In [ ]:
# pivot
df_fda_full = df_fda_full.pivot_table(values='Value', index=['FIPS', 'State', 'County'], columns='Variable_Code')
df_fda_full.reset_index(inplace=True)

In [ ]:
# display
df_fda_full.head()

Variable_Code,FIPS,State,County,2010_Census_Population,AGRITRSM_OPS07,AGRITRSM_OPS12,AGRITRSM_RCT07,AGRITRSM_RCT12,BERRY_ACRES07,BERRY_ACRES12,...,WICS16,WICSPTH11,WICSPTH16,WIC_PART_2012,WIC_PART_2013,WIC_PART_2014,WIC_PART_2015,WIC_PART_2016,WIC_PART_2017,WIC_PART_2018
0,1,AL,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.418998e+05,1.390005e+05,1.310462e+05,1.321328e+05,1.291599e+05,1.239926e+05,1.206048e+05
1,2,AK,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.496933e+04,2.305467e+04,1.960467e+04,1.968200e+04,1.912092e+04,1.818817e+04,1.709242e+04
2,4,AZ,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.932141e+05,1.825008e+05,1.730202e+05,1.670722e+05,1.639978e+05,1.535101e+05,1.495133e+05
3,5,AR,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.429275e+04,8.977683e+04,8.328858e+04,8.421958e+04,8.055467e+04,7.651850e+04,7.360675e+04
4,6,CA,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.472468e+06,1.431881e+06,1.348939e+06,1.265005e+06,1.174875e+06,1.080241e+06,1.009492e+06


In [ ]:
# shape
print(f'Shape of df_fda_full: {df_fda_full.shape}')

print('')

# columns
print(f'Columns in df_fda_full: {df_fda_full.columns}')

Shape of df_fda: (6336, 335)

Columns in df_fda: Index(['FIPS', 'State', 'County', '2010_Census_Population', 'AGRITRSM_OPS07',
       'AGRITRSM_OPS12', 'AGRITRSM_RCT07', 'AGRITRSM_RCT12', 'BERRY_ACRES07',
       'BERRY_ACRES12',
       ...
       'WICS16', 'WICSPTH11', 'WICSPTH16', 'WIC_PART_2012', 'WIC_PART_2013',
       'WIC_PART_2014', 'WIC_PART_2015', 'WIC_PART_2016', 'WIC_PART_2017',
       'WIC_PART_2018'],
      dtype='object', name='Variable_Code', length=335)


In [ ]:
# info
df_fda_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6336 entries, 0 to 6335
Columns: 335 entries, FIPS to WIC_PART_2018
dtypes: float64(332), int64(1), object(2)
memory usage: 16.2+ MB


In [ ]:
# describe
df_fda_full.describe()

Variable_Code,FIPS,2010_Census_Population,AGRITRSM_OPS07,AGRITRSM_OPS12,AGRITRSM_RCT07,AGRITRSM_RCT12,BERRY_ACRES07,BERRY_ACRES12,BERRY_ACRESPTH07,BERRY_ACRESPTH12,...,WICS16,WICSPTH11,WICSPTH16,WIC_PART_2012,WIC_PART_2013,WIC_PART_2014,WIC_PART_2015,WIC_PART_2016,WIC_PART_2017,WIC_PART_2018
count,6336.000000,3.142000e+03,3080.000000,3080.000000,1.974000e+03,2.119000e+03,2305.000000,2334.000000,2305.000000,2333.000000,...,2985.000000,3008.000000,2982.000000,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01
mean,30142.671559,9.826204e+04,7.579545,10.764935,2.108313e+05,2.766437e+05,108.856833,120.325621,1.728031,1.972856,...,14.577889,0.245190,0.217240,1.704073e+05,1.657204e+05,1.580896e+05,1.536727e+05,1.474524e+05,1.397460e+05,1.321477e+05
std,15342.995807,3.129467e+05,10.407053,15.438169,5.423634e+05,7.750182e+05,924.730823,975.864547,21.152581,22.985520,...,48.084899,0.211686,0.191315,2.468271e+05,2.411141e+05,2.287575e+05,2.180488e+05,2.065493e+05,1.932998e+05,1.797976e+05
min,1.000000,8.200000e+01,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.004029,0.004938,1.260158e+04,1.198017e+04,1.162892e+04,1.099542e+04,1.078283e+04,1.015858e+04,9.062000e+03
25%,18139.000000,1.111450e+04,2.000000,3.000000,7.000000e+03,1.300000e+04,0.000000,0.000000,0.000000,0.000000,...,2.000000,0.128155,0.114667,4.323908e+04,4.172146e+04,4.031725e+04,3.905363e+04,3.856721e+04,3.698321e+04,3.456450e+04
50%,29151.000000,2.587200e+04,5.000000,6.000000,4.200000e+04,6.500000e+04,5.000000,6.000000,0.101541,0.132127,...,5.000000,0.186860,0.165353,1.185849e+05,1.137628e+05,1.089005e+05,1.055036e+05,1.001519e+05,9.595817e+04,9.215250e+04
75%,45069.000000,6.678000e+04,9.000000,13.000000,1.687500e+05,2.330000e+05,22.000000,24.000000,0.400727,0.466003,...,10.000000,0.277210,0.247699,1.683068e+05,1.636985e+05,1.591861e+05,1.580745e+05,1.515955e+05,1.459558e+05,1.426490e+05
max,56045.000000,9.818605e+06,161.000000,236.000000,8.464000e+06,2.372300e+07,28904.000000,29218.000000,864.767832,898.490114,...,1188.000000,3.048780,3.262643,1.472468e+06,1.431881e+06,1.348939e+06,1.265005e+06,1.174875e+06,1.080241e+06,1.009492e+06


In [ ]:
df_fda_full.isnull().sum()

Variable_Code
FIPS                         0
State                        0
County                       0
2010_Census_Population    3194
AGRITRSM_OPS07            3256
                          ... 
WIC_PART_2014             6285
WIC_PART_2015             6285
WIC_PART_2016             6285
WIC_PART_2017             6285
WIC_PART_2018             6285
Length: 335, dtype: int64

In [50]:
df_fda_full

Variable_Code,FIPS,State,County,2010_Census_Population,AGRITRSM_OPS07,AGRITRSM_OPS12,AGRITRSM_RCT07,AGRITRSM_RCT12,BERRY_ACRES07,BERRY_ACRES12,...,WICS16,WICSPTH11,WICSPTH16,WIC_PART_2012,WIC_PART_2013,WIC_PART_2014,WIC_PART_2015,WIC_PART_2016,WIC_PART_2017,WIC_PART_2018
0,1001.0,AL,Autauga County,54571.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.418998e+05,1.390005e+05,1.310462e+05,1.321328e+05,1.291599e+05,1.239926e+05,1.206048e+05
1,1003.0,AL,Baldwin County,182265.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.496933e+04,2.305467e+04,1.960467e+04,1.968200e+04,1.912092e+04,1.818817e+04,1.709242e+04
2,1005.0,AL,Barbour County,27457.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.932141e+05,1.825008e+05,1.730202e+05,1.670722e+05,1.639978e+05,1.535101e+05,1.495133e+05
3,1007.0,AL,Bibb County,22915.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.429275e+04,8.977683e+04,8.328858e+04,8.421958e+04,8.055467e+04,7.651850e+04,7.360675e+04
4,1009.0,AL,Blount County,57322.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.472468e+06,1.431881e+06,1.348939e+06,1.265005e+06,1.174875e+06,1.080241e+06,1.009492e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6331,56041.0,WY,Uinta,NaN,5.0,9.0,105000.0,NaN,0.0,0.0,...,3.0,0.143589,0.144991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6332,56043.0,WY,Washakie County,8533.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6333,56043.0,WY,Washakie,NaN,8.0,6.0,70000.0,62000.0,0.0,NaN,...,2.0,0.236742,0.244858,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6334,56045.0,WY,Weston County,7208.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
col_fda = df_fda_full.columns.tolist()
for col in col_fda:
    print(col)

In [ ]:
df_fda_full.update(df_fda_county)

In [51]:
6336-3256 

3080

In [ ]:
count = 0
for col in df_fda_full.columns:
    if df_fda_full[col].isnull().sum() >= 3256:
        print(col)
        count +=1
    else:
        pass
        # print('not working')
print(count)
# all_cats[cat_list[i]] = df[df.Category_Code == cat_list[i]].Variable_Code.tolist()

#### df_fda

Here I'll create a new working dataframe, 'df_fda', which will feature only the relevant categories saved in 'cat_select'.

In [52]:
cat_select

['ACCESS', 'ASSISTANCE', 'INSECURITY', 'HEALTH', 'SOCIOECONOMIC']